In [3]:
filename = 'claims1.csv'
bgn = 2013
infl_2014 = 1.05
infl_2015 = 1.06
infl_2016 = 1.07
infl_2017 = 1.05
infl_future = 1.1

In [24]:
def chainladder(filename,bgn, infla_1, infla_2, infla_3, infla_4, infla_future) :   
    import numpy as np
    import pandas as pd
    cum_claims = pd.read_csv(filename)
    cum_claims.set_index('Year', inplace =True)
    noncum_claims = cum_claims.copy()
    
    for Y in range(0,5):
        for I in  range(1,5):
            noncum_claims.iloc[Y,I] = cum_claims.iloc[Y,I] - cum_claims.iloc[Y,I-1]
    
    inflation = pd.DataFrame(np.array([[infl_2014*infl_2015*infl_2016*infl_2017], [infl_2015*infl_2016*infl_2017], [infl_2016*infl_2017],[infl_2017]])
                                  ,columns=['to 2017'], index = ['from 2013', 'from 2014', 'from 2015', 'from 2016'])
    
    noncum_infla0 = noncum_claims.copy()
    for Y in range(0,5):
        for I in range(0,5):
            year = Y+bgn+I
            t = Y+I
            if year<2017 :
                noncum_infla0.iloc[Y,I] = noncum_infla0.iloc[Y,I]*inflation.iloc[t,0]
        
    cum_infla = noncum_infla0.copy()
    for Y in range(0,5):
        for I in  range(1,5):
            cum_infla.iloc[Y,I] = noncum_infla0.iloc[Y,I] + cum_infla.iloc[Y,I-1]
    
    lambda_i = []
    for I in range(1,5):
        count = cum_infla.iloc[:,I].count()
        lamb = cum_infla.iloc[0:count,I].sum() / cum_infla.iloc[0:count,I-1].sum()
        lambda_i.append(lamb)
    
    while cum_infla.isnull().sum().sum() != 0:
        for Y in range(1,5):
            if cum_infla.iloc[Y,:].isnull().any() == True:
                count = cum_infla.iloc[Y,:].count()
                cum_infla.iloc[Y,count] = cum_infla.iloc[Y,count-1] * lambda_i[count-1]
            
    noncum_infla = cum_infla.copy()
    for Y in range(0,5):
        for I in  range(1,5):
            noncum_infla.iloc[Y,I] = cum_infla.iloc[Y,I] - cum_infla.iloc[Y,I-1]
        
    for Y in range(0,5):
        for I in range(0,5):
            year = Y+bgn+I
            t = year-2017
            noncum_infla.iloc[Y,I] = noncum_infla.iloc[Y,I]*(infl_future**t)
    
    amount_pay = pd.DataFrame(np.array([[0], [0], [0], [0], [0]])
                        ,columns = ['amount pay'], index = ['2018','2019','2020','2021', 'Total'])
    for y in range(0,5):
        for i in range(0,5):
            year_future = bgn + y + i
            t = year_future - 2018
            if year_future > 2017 :
                amount_pay.iloc[t,0] = amount_pay.iloc[t,0] + noncum_infla.iloc[y,i]      
    for i in range(0,4):
        amount_pay.iloc[4,0] = amount_pay.iloc[4,0] + amount_pay.iloc[i,0]

    reserve = pd.DataFrame(np.array([[0], [0], [0], [0], [0]])
                        ,columns = ['reserve needed'], index = ['2014','2015','2016','2017', 'Total'])
    for y in range(0,5):
        for i in range(0,5):
            year_current = bgn + y
            year_future = bgn + y + i
            if year_future > 2017 :
                reserve.iloc[y-1,0] = reserve.iloc[y-1,0] + noncum_infla.iloc[y,i]
    for i in range(0,4):
        reserve.iloc[4,0] = reserve.iloc[4,0] + reserve.iloc[i,0]
    
    print('Based Cumulative claims')
    display(cum_claims)
    print('Based Incremental claims')
    display(noncum_claims)
    print('Forcasted Inflation-adjusted Cumulative claims')
    display(cum_infla)
    print('Forcasted Inflation-adjusted Incremental claims')
    display(noncum_infla)
    print('Amount inserer will pay')
    display(amount_pay)
    print('Reserve needed')
    display(reserve)

In [25]:
chainladder(filename, bgn, infl_2014, infl_2015, infl_2015, infl_2016, infl_future) 

Based Cumulative claims


,0,1,2,3,4
Year,,,,,
2013,880,988.0,1046.0,1065.0,1262.0
2014,940,1034.0,1091.0,1095.0,NaN
2015,1060,1161.0,1229.0,NaN,NaN
2016,1120,1221.0,NaN,NaN,NaN
2017,1240,NaN,NaN,NaN,NaN


Based Incremental claims


,0,1,2,3,4
Year,,,,,
2013,880,108.0,58.0,19.0,197.0
2014,940,94.0,57.0,4.0,NaN
2015,1060,101.0,68.0,NaN,NaN
2016,1120,101.0,NaN,NaN,NaN
2017,1240,NaN,NaN,NaN,NaN


Forcasted Inflation-adjusted Cumulative claims


...

,0,1,2,3,4
Year,,,,,
2013,751.588580,96.632817,53.853719,18.136364,197.000000
2014,841.063411,87.280165,54.409091,4.000000,212.541605
2015,984.223140,96.409091,68.000000,13.942817,249.888819
2016,1069.090909,101.000000,72.279983,15.087081,270.396776
2017,1240.000000,131.225831,84.631869,17.665304,316.604733


Amount inserer will pay


,amount pay
2018,429.990236
2019,349.607769
2020,288.062080
2021,316.604733
Total,1384.264819


Reserve needed


,reserve needed
2014,212.541605
2015,263.831636
2016,357.763840
2017,550.127737
Total,1384.264819
